In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("udf").getOrCreate()

24/04/16 17:07:54 WARN Utils: Your hostname, MZC01-HYUCKSANGCHO.local resolves to a loopback address: 127.0.0.1; using 10.90.9.242 instead (on interface en0)
24/04/16 17:07:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/16 17:07:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/04/16 17:07:55 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
transactions = [
    ('찹쌀탕수육+짜장2', '2021-11-07 13:20:00', 22000, 'KRW'),
    ('등심탕수육+크립새우+짜장면', '2021-10-24 11:19:00', 21500, 'KRW'), 
    ('월남 쌈 2인 세트', '2021-07-25 11:12:40', 42000, 'KRW'), 
    ('콩국수+열무비빔국수', '2021-07-10 08:20:00', 21250, 'KRW'), 
    ('장어소금+고추장구이', '2021-07-01 05:36:00', 68700, 'KRW'), 
    ('족발', '2020-08-19 19:04:00', 32000, 'KRW'),  
]

schema = ["name", "datetime", "price", "currency"]
df = spark.createDataFrame(data=transactions, schema=schema)

In [4]:
df.show()

+--------------------------+-------------------+-----+--------+
|                      name|           datetime|price|currency|
+--------------------------+-------------------+-----+--------+
|          찹쌀탕수육+짜장2|2021-11-07 13:20:00|22000|     KRW|
|등심탕수육+크립새우+짜장면|2021-10-24 11:19:00|21500|     KRW|
|          월남 쌈 2인 세트|2021-07-25 11:12:40|42000|     KRW|
|       콩국수+열무비빔국수|2021-07-10 08:20:00|21250|     KRW|
|       장어소금+고추장구이|2021-07-01 05:36:00|68700|     KRW|
|                      족발|2020-08-19 19:04:00|32000|     KRW|
+--------------------------+-------------------+-----+--------+



In [9]:
df.createOrReplaceTempView("transactions")

In [11]:
spark.sql("select * from transactions").show()

+--------------------------+-------------------+-----+--------+
|                      name|           datetime|price|currency|
+--------------------------+-------------------+-----+--------+
|          찹쌀탕수육+짜장2|2021-11-07 13:20:00|22000|     KRW|
|등심탕수육+크립새우+짜장면|2021-10-24 11:19:00|21500|     KRW|
|          월남 쌈 2인 세트|2021-07-25 11:12:40|42000|     KRW|
|       콩국수+열무비빔국수|2021-07-10 08:20:00|21250|     KRW|
|       장어소금+고추장구이|2021-07-01 05:36:00|68700|     KRW|
|                      족발|2020-08-19 19:04:00|32000|     KRW|
+--------------------------+-------------------+-----+--------+



In [16]:
from pyspark.sql.types import LongType
def squared(n):
    return n ** 2

spark.udf.register("squared", squared, LongType())

24/04/16 17:14:23 WARN SimpleFunctionRegistry: The function squared replaced a previously registered function.


<function __main__.squared(n)>

In [17]:
spark.sql("select price, squared(price) from transactions").show()

+-----+--------------+
|price|squared(price)|
+-----+--------------+
|22000|     484000000|
|21500|     462250000|
|42000|    1764000000|
|21250|     451562500|
|68700|    4719690000|
|32000|    1024000000|
+-----+--------------+



In [18]:
spark.sql("select price, squared(price) from transactions").printSchema()

root
 |-- price: long (nullable = true)
 |-- squared(price): long (nullable = true)



In [19]:
from pyspark.sql.functions import udf

@udf("long")
def squared(n):
    return n ** 2

spark.sql("select price, squared(price) from transactions").show()

+-----+--------------+
|price|squared(price)|
+-----+--------------+
|22000|     484000000|
|21500|     462250000|
|42000|    1764000000|
|21250|     451562500|
|68700|    4719690000|
|32000|    1024000000|
+-----+--------------+



In [22]:
def read_number(n):
    units = ["", "십", "백", "천", "만"]
    nums = '일이삼사오육칠팔구'
    i = 0
    res = []
    while n > 0:
        n,r = divmod(n, 10)
        if r > 0:
            res.append(nums[r-1] + units[i])
        i += 1
    return "".join(reversed(res))

print(read_number(21250))
print(read_number(68700))

이만일천이백오십
육만팔천칠백


In [23]:
spark.udf.register("read_number", read_number)

<function __main__.read_number(n)>

In [24]:
spark.sql("select price, read_number(price) from transactions").show()

+-----+------------------+
|price|read_number(price)|
+-----+------------------+
|22000|          이만이천|
|21500|      이만일천오백|
|42000|          사만이천|
|21250|  이만일천이백오십|
|68700|      육만팔천칠백|
|32000|          삼만이천|
+-----+------------------+



24/04/16 21:34:50 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1043298 ms exceeds timeout 120000 ms
24/04/16 21:34:50 WARN SparkContext: Killing executors is not supported by current scheduler.
24/04/16 21:51:04 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:322)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:641)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1111)
	at org.apache.spark.executor.Executor.$anonfun$heartbeater$1(Executor.scala:244)
	at s